In [1]:
from bqplot import LinearScale, OrdinalColorScale, ColorAxis, Axis, Lines, Figure
from ipywidgets import HBox, VBox, SelectionSlider, jslink, jsdlink, Layout, IntText

In [2]:
%%html
<style>
    /* prevent truncation of the slider labels */
    .widget-label, .widget-button, .jupyter-button, .widget-readout {
        width: unset !important;
        min-width: fit-content !important;
    }

    .widget-label {
        font-weight: bolder;
    }
</style>

In [3]:
sampling_target = 10
initial_last_harvest_count = 100
sampledTrue_counts = range(0, 21)

In [4]:
def calculate_exponential_backoff(previous_harvest_count):
    results = {
        'targets': [],
        'min_priorities': [],
        'probabilities': []
    }

    for sampledTrue_count in sampledTrue_counts:
        target = sampling_target

        if sampledTrue_count > target:
            ratio = target / sampledTrue_count

            # the "exponential" from the name - and note that this is Ruby's implementation!
            target = target ** ratio - target ** 0.5

        probability = target / max(sampling_target, previous_harvest_count)

        results['targets'].append(target)
        results['probabilities'].append(probability)
        results['min_priorities'].append(1.0 - probability)
        
    return results

In [5]:
x_sc = LinearScale(min=0, max=20)
ny_sc = LinearScale(min=0, max=10)
my_sc = LinearScale(min=0, max=1)
py_sc = LinearScale(min=0, max=1)

ax_x = Axis(label='Number of sampled=True transactions this Harvest',
             scale=x_sc, grid_lines='solid')
nax_y = Axis(label='Next Sampling Target', scale=ny_sc, side='left', grid_lines='solid',
             orientation='vertical', label_color='blue')
pax_y = Axis(label='Probability Next Transaction Will be Sampled', scale=py_sc, side='left', grid_lines='solid',
             orientation='vertical', label_color='red')
max_y = Axis(label='Minimum Priority', scale=my_sc, side='right', grid_lines='solid',
             orientation='vertical', label_color='green')

In [6]:
initial_results = calculate_exponential_backoff(initial_last_harvest_count)

count_line = Lines(
    x=sampledTrue_counts,
    y=initial_results['targets'],
    colors=['blue'],
    scales={'x': x_sc, 'y': ny_sc})

min_priority_line = Lines(
    x=sampledTrue_counts,
    y=initial_results['min_priorities'],
    colors=['green'],
    scales={'x': x_sc, 'y': my_sc})

probabilities_line = Lines(
    x=sampledTrue_counts,
    y=initial_results['probabilities'],
    colors=['red'],
    scales={'x': x_sc, 'y': py_sc})

In [7]:
lhcs = SelectionSlider(
    options=[1, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000],
    value=initial_last_harvest_count,
    description='Last Harvest Count:'
)

def update_plots(slider):
    previous_harvest_count = lhcs.value
    results = calculate_exponential_backoff(previous_harvest_count)
    
    count_line.y = results['targets']
    min_priority_line.y = results['min_priorities']
    probabilities_line.y = results['probabilities']

lhcs.observe(update_plots, names='value')

In [8]:
fig1 = Figure(
    marks=[count_line], axes=[ax_x, nax_y],
    title='Exponential Backoff of Sampling Target')

fig2 = Figure(
    marks=[min_priority_line, probabilities_line], axes=[ax_x, max_y, pax_y],
    title='Effect of Exponential Backoff',
    legend_location='right')

VBox([
    HBox([lhcs], layout=Layout(margin='auto')),
    HBox([fig1, fig2])
])